# Tweet Sentiment Analysis with Emoji Support

This notebook implements a complete sentiment analysis pipeline for tweets that can handle both text and emojis.

## Dataset Overview:
- **Sentiment140**: 1.6M labeled tweets (positive/negative)
- **Emoji Sentiment Data**: Emoji sentiment scores from social media analysis

## 1. Import Libraries and Setup

In [2]:
# Install required packages
%pip install emoji scikit-learn matplotlib seaborn

  Using cached joblib-1.5.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/590.6 kB ? eta -:--:--
   --------------------------------------- 590.6/590.6 kB 10.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   ---------------------------- ----------- 6.3/8.7 MB 32.2 MB/s eta 0:00:01
   ---------------------------------------- 8.7/8.7 MB 30.2 MB/s eta 0:00:00
Using cached joblib-1.5.1-py3-none-any.whl (307 kB)
   ---------------------------------------- 0.0/38.5 MB ? eta -:--:--
   -------- ------------------------------- 8.7/38.5 MB 41.3 MB/s eta 0:00:01
   ----------------- ---------------------- 16.8/38.5 MB 39.1 MB/s eta 0:00:01
   ------------------------- -------------- 24.9/38.5 MB 39.4 MB/s eta 0:00:01
   ---------------------------------- ----- 33.0/38.5 MB 39.6 MB/s eta 0:00:01
   ---------------------------------------  38.3/38.5 MB 

In [3]:
import pandas as pd
import numpy as np
import re
import pickle
import emoji
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

## 2. Load and Explore Datasets

In [17]:
pd.set_option('display.max_columns', None)

In [18]:
# Load Sentiment140 dataset
# Format: sentiment(0=negative, 4=positive), id, date, query, user, text
columns = ['sentiment', 'id', 'date', 'query', 'user', 'text']
sentiment140_df = pd.read_csv('data/tweets/training.1600000.processed.noemoticon.csv', 
                             names=columns, encoding='latin-1')

print(f"Sentiment140 dataset shape: {sentiment140_df.shape}")
print(f"Sentiment distribution:\n{sentiment140_df['sentiment'].value_counts()}")
sentiment140_df.head()

Sentiment140 dataset shape: (1600000, 6)
Sentiment distribution:
sentiment
0    800000
4    800000
Name: count, dtype: int64


,sentiment,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [19]:
# Load emoji sentiment data
emoji_df = pd.read_csv('data/emoji/Emoji_Sentiment_Data_v1.0.csv')
print(f"Emoji dataset shape: {emoji_df.shape}")
emoji_df.head()

Emoji dataset shape: (969, 9)


,Emoji,Unicode codepoint,Occurrences,Position,Negative,Neutral,Positive,Unicode name,Unicode block
0,😂,0x1f602,14622,0.805101,3614,4163,6845,FACE WITH TEARS OF JOY,Emoticons
1,❤,0x2764,8050,0.746943,355,1334,6361,HEAVY BLACK HEART,Dingbats
2,♥,0x2665,7144,0.753806,252,1942,4950,BLACK HEART SUIT,Miscellaneous Symbols
3,😍,0x1f60d,6359,0.765292,329,1390,4640,SMILING FACE WITH HEART-SHAPED EYES,Emoticons
4,😭,0x1f62d,5526,0.803352,2412,1218,1896,LOUDLY CRYING FACE,Emoticons


## 3. Data Preprocessing

In [6]:
# Create emoji sentiment mapping
def create_emoji_sentiment_map(emoji_df):
    """
    Create a mapping of emojis to sentiment scores
    Score calculation: (Positive - Negative) / Total occurrences
    """
    emoji_sentiment = {}
    
    for _, row in emoji_df.iterrows():
        emoji_char = row['Emoji']
        positive = row['Positive']
        negative = row['Negative']
        neutral = row['Neutral']
        total = positive + negative + neutral
        
        if total > 0:
            # Sentiment score: -1 (very negative) to +1 (very positive)
            sentiment_score = (positive - negative) / total
            emoji_sentiment[emoji_char] = sentiment_score
    
    return emoji_sentiment

emoji_sentiment_map = create_emoji_sentiment_map(emoji_df)
print(f"Created emoji sentiment mapping for {len(emoji_sentiment_map)} emojis")

# Show some examples
sorted_emojis = sorted(emoji_sentiment_map.items(), key=lambda x: x[1], reverse=True)
print("\nMost positive emojis:")
for emoji_char, score in sorted_emojis[:5]:
    print(f"{emoji_char}: {score:.3f}")
    
print("\nMost negative emojis:")
for emoji_char, score in sorted_emojis[-5:]:
    print(f"{emoji_char}: {score:.3f}")

Created emoji sentiment mapping for 969 emojis

Most positive emojis:
┊: 1.000
▃: 1.000
🔅: 1.000
🏧: 1.000
🔔: 1.000

Most negative emojis:
🕔: -1.000
🈂: -1.000
🎰: -1.000
҂: -1.000
╤: -1.000


In [7]:
def preprocess_text(text, emoji_sentiment_map):
    """
    Preprocess tweet text with emoji handling
    """
    if pd.isna(text):
        return ""
    
    # Convert to string and lowercase
    text = str(text).lower()
    
    # Extract emoji sentiment features
    emoji_sentiment_score = 0
    emoji_count = 0
    
    # Find emojis and calculate sentiment
    for char in text:
        if char in emoji_sentiment_map:
            emoji_sentiment_score += emoji_sentiment_map[char]
            emoji_count += 1
    
    # Replace emojis with sentiment indicators
    for emoji_char, sentiment_score in emoji_sentiment_map.items():
        if emoji_char in text:
            if sentiment_score > 0.2:
                text = text.replace(emoji_char, ' POSITIVE_EMOJI ')
            elif sentiment_score < -0.2:
                text = text.replace(emoji_char, ' NEGATIVE_EMOJI ')
            else:
                text = text.replace(emoji_char, ' NEUTRAL_EMOJI ')
    
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # Remove user mentions and hashtags (but keep the text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#', '', text)
    
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Test the preprocessing function
test_text = "I love this! 😍❤️ But I'm also sad 😭 #happy @user"
processed = preprocess_text(test_text, emoji_sentiment_map)
print(f"Original: {test_text}")
print(f"Processed: {processed}")

Original: I love this! 😍❤️ But I'm also sad 😭 #happy @user
Processed: i love this! POSITIVE_EMOJI POSITIVE_EMOJI ️ but i'm also sad NEUTRAL_EMOJI happy


In [8]:
# Prepare the dataset
# Convert sentiment labels: 0 -> 0 (negative), 4 -> 1 (positive)
sentiment140_df['label'] = sentiment140_df['sentiment'].map({0: 0, 4: 1})

# Use a subset for faster training
sample_size = 200000  # Use 200k tweets for training
df_sample = sentiment140_df.sample(n=sample_size, random_state=42).copy()

# Preprocess tweets
print("Preprocessing tweets...")
df_sample['processed_text'] = df_sample['text'].apply(lambda x: preprocess_text(x, emoji_sentiment_map))

# Remove empty texts
df_sample = df_sample[df_sample['processed_text'].str.len() > 0]

print(f"Final dataset shape: {df_sample.shape}")
print(f"Label distribution:\n{df_sample['label'].value_counts()}")

Preprocessing tweets...
Final dataset shape: (199631, 8)
Label distribution:
label
1    99972
0    99659
Name: count, dtype: int64


## 4. Feature Engineering and Model Training

In [9]:
# Split the data
X = df_sample['processed_text']
y = df_sample['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Training set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")
print(f"Training label distribution:\n{y_train.value_counts()}")

Training set size: 159704
Test set size: 39927
Training label distribution:
label
1    79977
0    79727
Name: count, dtype: int64


In [10]:
# Create and train the model pipeline
print("Training Logistic Regression model...")

# TF-IDF Vectorizer with optimized parameters
tfidf = TfidfVectorizer(
    max_features=50000,  # Limit vocabulary size
    ngram_range=(1, 2),  # Use unigrams and bigrams
    min_df=2,           # Ignore terms that appear in less than 2 documents
    max_df=0.95,        # Ignore terms that appear in more than 95% of documents
    strip_accents='unicode',
    stop_words='english'
)

# Logistic Regression model
lr_model = LogisticRegression(
    random_state=42,
    max_iter=1000,
    C=1.0
)

# Create pipeline
sentiment_pipeline = Pipeline([
    ('tfidf', tfidf),
    ('classifier', lr_model)
])

# Train the model
sentiment_pipeline.fit(X_train, y_train)

print("Model training completed!")

Training Logistic Regression model...
Model training completed!


## 5. Model Evaluation

In [11]:
# Make predictions
y_pred = sentiment_pipeline.predict(X_test)
y_pred_proba = sentiment_pipeline.predict_proba(X_test)[:, 1]

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Detailed classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Negative', 'Positive']))

Accuracy: 0.7718

Classification Report:
              precision    recall  f1-score   support

    Negative       0.78      0.75      0.77     19932
    Positive       0.76      0.79      0.78     19995

    accuracy                           0.77     39927
   macro avg       0.77      0.77      0.77     39927
weighted avg       0.77      0.77      0.77     39927



Detailed Metrics:
Precision: 0.7608
Recall: 0.7939
F1-Score: 0.7770
True Negatives: 14941
False Positives: 4991
False Negatives: 4121
True Positives: 15874


In [12]:
# Confusion Matrix
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Negative', 'Positive'],
            yticklabels=['Negative', 'Positive'])
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

# Calculate additional metrics
tn, fp, fn, tp = cm.ravel()
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2 * (precision * recall) / (precision + recall)

print(f"\nDetailed Metrics:")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1_score:.4f}")
print(f"True Negatives: {tn}")
print(f"False Positives: {fp}")
print(f"False Negatives: {fn}")
print(f"True Positives: {tp}")

## 6. Save the Model

In [13]:
# Save the trained model and emoji sentiment mapping
with open('tweet_sentiment_model.pkl', 'wb') as f:
    pickle.dump(sentiment_pipeline, f)

with open('emoji_sentiment_map.pkl', 'wb') as f:
    pickle.dump(emoji_sentiment_map, f)

print("Model and emoji mapping saved successfully!")
print("Files saved:")
print("- tweet_sentiment_model.pkl")
print("- emoji_sentiment_map.pkl")

Model and emoji mapping saved successfully!
Files saved:
- tweet_sentiment_model.pkl
- emoji_sentiment_map.pkl


## 7. Create Prediction Function

In [14]:
def predict_tweet_sentiment(tweet, model_pipeline, emoji_map):
    """
    Predict sentiment for a single tweet
    
    Args:
        tweet (str): The tweet text
        model_pipeline: Trained sentiment analysis pipeline
        emoji_map (dict): Emoji sentiment mapping
    
    Returns:
        dict: Prediction results with sentiment, confidence, and emoji analysis
    """
    # Preprocess the tweet
    processed_tweet = preprocess_text(tweet, emoji_map)
    
    # Make prediction
    prediction = model_pipeline.predict([processed_tweet])[0]
    confidence = model_pipeline.predict_proba([processed_tweet])[0]
    
    # Analyze emojis in the original tweet
    emoji_analysis = []
    for char in tweet:
        if char in emoji_map:
            emoji_analysis.append({
                'emoji': char,
                'sentiment_score': emoji_map[char]
            })
    
    return {
        'original_tweet': tweet,
        'processed_tweet': processed_tweet,
        'sentiment': 'Positive' if prediction == 1 else 'Negative',
        'confidence': {
            'negative': confidence[0],
            'positive': confidence[1]
        },
        'emoji_analysis': emoji_analysis
    }

# Test the prediction function
test_tweets = [
    "I love this movie! 😍❤️ It's absolutely amazing!",
    "This is terrible 😭💔 I hate it so much",
    "Just had lunch 🍕 it was okay I guess",
    "Beautiful sunset today! 🌅✨ Perfect weather for a walk",
    "Stuck in traffic again 😤🚗 This is so frustrating!"
]

print("Testing the model with sample tweets:\n")
for i, tweet in enumerate(test_tweets, 1):
    result = predict_tweet_sentiment(tweet, sentiment_pipeline, emoji_sentiment_map)
    print(f"Tweet {i}: {result['original_tweet']}")
    print(f"Sentiment: {result['sentiment']}")
    print(f"Confidence: {result['confidence']['positive']:.3f} (positive), {result['confidence']['negative']:.3f} (negative)")
    if result['emoji_analysis']:
        print("Emojis found:")
        for emoji_info in result['emoji_analysis']:
            print(f"  {emoji_info['emoji']}: {emoji_info['sentiment_score']:.3f}")
    print("-" * 50)

Testing the model with sample tweets:

Tweet 1: I love this movie! 😍❤️ It's absolutely amazing!
Sentiment: Positive
Confidence: 0.847 (positive), 0.153 (negative)
Emojis found:
  😍: 0.678
  ❤: 0.746
--------------------------------------------------
Tweet 2: This is terrible 😭💔 I hate it so much
Sentiment: Negative
Confidence: 0.112 (positive), 0.888 (negative)
Emojis found:
  😭: -0.093
  💔: -0.122
--------------------------------------------------
Tweet 3: Just had lunch 🍕 it was okay I guess
Sentiment: Positive
Confidence: 0.551 (positive), 0.449 (negative)
Emojis found:
  🍕: 0.420
--------------------------------------------------
Tweet 4: Beautiful sunset today! 🌅✨ Perfect weather for a walk
Sentiment: Positive
Confidence: 0.825 (positive), 0.175 (negative)
Emojis found:
  🌅: 0.381
  ✨: 0.353
--------------------------------------------------
Tweet 5: Stuck in traffic again 😤🚗 This is so frustrating!
Sentiment: Negative
Confidence: 0.039 (positive), 0.961 (negative)
Emojis found:
 

## 8. Model Performance Summary

In [15]:
# Feature importance (top words)
feature_names = sentiment_pipeline.named_steps['tfidf'].get_feature_names_out()
coefficients = sentiment_pipeline.named_steps['classifier'].coef_[0]

# Get top positive and negative features
top_positive_idx = np.argsort(coefficients)[-20:]
top_negative_idx = np.argsort(coefficients)[:20]

print("Top 20 words indicating POSITIVE sentiment:")
for idx in reversed(top_positive_idx):
    print(f"{feature_names[idx]}: {coefficients[idx]:.4f}")

print("\nTop 20 words indicating NEGATIVE sentiment:")
for idx in top_negative_idx:
    print(f"{feature_names[idx]}: {coefficients[idx]:.4f}")

Top 20 words indicating POSITIVE sentiment:
thanks: 5.1621
welcome: 5.0186
thank: 4.6277
wish luck: 4.4758
glad: 4.1019
awesome: 4.0078
congratulations: 3.8601
yay: 3.7581
hehe: 3.7423
excited: 3.5737
great: 3.4268
amazing: 3.4036
love: 3.3766
smiling: 3.3477
happy: 3.3156
proud: 3.3146
smile: 3.3121
congrats: 3.3038
cute: 3.1989
beautiful: 3.1297

Top 20 words indicating NEGATIVE sentiment:
sad: -11.0857
missing: -6.5680
sick: -6.5674
poor: -6.4339
miss: -6.2508
sucks: -6.0347
hate: -5.8727
wish: -5.8568
bummed: -5.5240
hurts: -5.4990
unfortunately: -5.3635
lost: -5.0281
disappointed: -5.0103
ugh: -4.9875
sorry: -4.9567
depressing: -4.8878
died: -4.8396
bad: -4.8391
rip: -4.8356
headache: -4.7252


In [16]:
# Final model summary
print("="*60)
print("TWEET SENTIMENT ANALYSIS MODEL SUMMARY")
print("="*60)
print(f"Training Data: {len(X_train):,} tweets")
print(f"Test Data: {len(X_test):,} tweets")
print(f"Vocabulary Size: {len(feature_names):,} features")
print(f"Emoji Mappings: {len(emoji_sentiment_map)} emojis")
print(f"\nModel Performance:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1_score:.4f}")
print(f"\nModel saved as: tweet_sentiment_model.pkl")
print(f"Emoji mapping saved as: emoji_sentiment_map.pkl")
print("\nThe model can now classify tweets with emojis as positive or negative!")

TWEET SENTIMENT ANALYSIS MODEL SUMMARY
Training Data: 159,704 tweets
Test Data: 39,927 tweets
Vocabulary Size: 50,000 features
Emoji Mappings: 969 emojis

Model Performance:
Accuracy: 0.7718
Precision: 0.7608
Recall: 0.7939
F1-Score: 0.7770

Model saved as: tweet_sentiment_model.pkl
Emoji mapping saved as: emoji_sentiment_map.pkl

The model can now classify tweets with emojis as positive or negative!
